In [8]:
#Importar librerias
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sympy import diff,symbols,dsolve, expand_complex,Matrix, pi, cos, sin, simplify, eye, solve, latex, atan2, asin,pprint, init_printing, Derivative, sqrt, integrate, Eq, lambdify
from sympy.physics.mechanics import ReferenceFrame, dynamicsymbols, init_vprinting
import sympy
# Configuración de impresión en formato LaTeX (MathJax)
init_vprinting(use_latex='mathjax')

In [ ]:
N1 = ReferenceFrame("N")
x, theta = dynamicsymbols("x, theta") #como definimos theta, entonces crece con el momento, asi que theta doble punto es alfa
N2 = N1.orientnew("N2", "Axis", (-np.deg2rad(30), N1.z)) #Slider
A = N1.orientnew("A", "Axis", (theta, N1.z)) #T
A.x.express(N1).subs(theta,0) #Si theta es 0, apunta como el normal

l,m,g,t = symbols("l,m,g,t")
lcm = 3*l/4
Icm = 0.2*l # falta calcularlohacemos dos inercias, entonces es la inercia de una horizontal, mas la inercia de una vertical, la inercia vertical por ejes paralelos toca moverlas al punto de centro de masa

#Cinematica: Expresar 
rcm = x*N2.x - lcm*A.x
acm = rcm.diff(t,N1).diff(t,N1) #derivada de un VECTOR
acm
alpha = theta.diff(t,t) #diff(t,2) = diff(t,t) = diff(t,).diff(t) derivada de un ESCALAR

#Eq. Newton Euler
#No hay momentos porque no tiene masa
F_n = symbols("F_n")
sumF = -F_n*N2.y - m*g*N1.y -m*acm
sumM = (lcm*A.x).cross(-F_n*N2.y)-Icm*alpha*N1.z #F_n produce momentos porque está alineado con la barra, si theta no es 60 (complementario)

eqs = [sumF.dot(N1.x), sumF.dot(N1.y), sumM.dot(N1.z)]

AA , b = sympy.linear_eq_to_matrix(eqs, [x.diff(t,t), theta.diff(t,t), F_n])

AA #solo depende de la posicion actual del sistema, hay términos que no dependen del ángulo, como la primera columna
#Aqui no aparece theta punto, solo en b
b #terminos que no factorizan en las incognitas, terminos indepeneidnetes, el a_cm = -l*theta_dot**2

#Despejar ecuaciones
#params = {l:1, m:1, g:9.81 }

#EN el estado actual, 
ic = {theta:0, theta.diff(t):0}
AA.subs(ic)
b.subs(ic)

#AA.subs(ic).inv()*b.subs(ic) #vector resultante para la condición inicial
AA_fun = sympy.lambdify([x,x.diff(t), theta, theta.diff(t)], AA)
b_fun = sympy.lambdify([x,x.diff(t), theta, theta.diff(t)], b)


def odefun(q,t=None):
    x,xdot,theta,thetadot = q
    AA_num = AA_fun(x,xdot,theta,thetadot)
    b_num = b_fun(x,xdot,theta,thetadot)
    print(AA_num)
    print(b_num)
    
odefun([0,0,0,0])

⎡        0.1⋅g⋅m        ⎤
⎢ ───────────────────── ⎥
⎢             2         ⎥
⎢ 0.421875⋅l⋅m  + 0.2⋅m ⎥
⎢                       ⎥
⎢      0.5625⋅g⋅m       ⎥
⎢  ──────────────────   ⎥
⎢  0.421875⋅l⋅m + 0.2   ⎥
⎢                       ⎥
⎢-0.173205080756888⋅g⋅m ⎥
⎢───────────────────────⎥
⎣  0.421875⋅l⋅m + 0.2   ⎦